# Coconut Mite Detection Model - Training Results

**Model:** MobileNetV2 (Transfer Learning)  
**Dataset:** Pre-organized with NO data leakage  
**Version:** v6

## 1. Setup and Imports

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

## 2. Configuration

In [ ]:
# Paths
MODEL_DIR = Path(r"D:\SLIIT\Reaserch Project\CoconutHealthMonitor\Research\ml\models\coconut_mite_v6")
DATA_DIR = Path(r"D:\SLIIT\Reaserch Project\CoconutHealthMonitor\Research\ml\data\raw\pest")

# Settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
CLASS_NAMES = ['coconut_mite', 'healthy']

print(f"Model Directory: {MODEL_DIR}")
print(f"Data Directory: {DATA_DIR}")
print(f"Image Size: {IMG_SIZE}")
print(f"Classes: {CLASS_NAMES}")

## 3. Dataset Distribution

In [ ]:
# Count images in each split
def count_images(data_dir, class_names):
    counts = {}
    splits_map = {
        'Train': 'train',
        'Validation': 'validation', 
        'Test': 'test'
    }
    
    for split_name, split_key in splits_map.items():
        counts[split_key] = {}
        for cls in class_names:
            # Try different folder names
            for folder in [split_name, split_name.lower(), 'test', 'Test']:
                cls_dir = data_dir / cls / folder
                if cls_dir.exists():
                    count = len([f for f in cls_dir.glob('*.*') 
                                if f.suffix.lower() in ['.jpg', '.jpeg', '.png']])
                    counts[split_key][cls] = count
                    break
            if cls not in counts[split_key]:
                counts[split_key][cls] = 0
    return counts

counts = count_images(DATA_DIR, CLASS_NAMES)

print("="*60)
print("DATASET DISTRIBUTION (NO Data Leakage)")
print("="*60)
print(f"\n{'Split':<15} {'Mite':<10} {'Healthy':<10} {'Total':<10}")
print("-"*45)
total_all = 0
for split in ['train', 'validation', 'test']:
    mite = counts[split].get('coconut_mite', 0)
    healthy = counts[split].get('healthy', 0)
    total = mite + healthy
    total_all += total
    note = "(augmented)" if split == 'train' else "(originals)"
    print(f"{split.capitalize():<15} {mite:<10} {healthy:<10} {total:<10} {note}")
print("-"*45)
print(f"{'TOTAL':<15} {'':<10} {'':<10} {total_all:<10}")

In [ ]:
# Visualize distribution
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
colors = ['#e74c3c', '#2ecc71']  # Red for mite, green for healthy

for ax, split in zip(axes, ['train', 'validation', 'test']):
    vals = [counts[split].get(c, 0) for c in CLASS_NAMES]
    total = sum(vals)
    bars = ax.bar(CLASS_NAMES, vals, color=colors, edgecolor='black', linewidth=1.5)
    
    title_suffix = "(augmented)" if split == 'train' else "(originals)"
    ax.set_title(f'{split.capitalize()} Set\n{total} images {title_suffix}', 
                 fontsize=11, fontweight='bold')
    ax.set_ylabel('Number of Images')
    ax.set_ylim([0, max(vals) * 1.2 if max(vals) > 0 else 10])
    
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x() + bar.get_width()/2, v + max(vals)*0.02, 
                f'{v}', ha='center', fontweight='bold', fontsize=12)

plt.suptitle('Dataset Distribution (NO Data Leakage)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\nKey Points:")
print("  - Training set contains augmented images")
print("  - Validation and Test sets contain ONLY original images")
print("  - NO overlap between splits (no data leakage)")

## 4. Load Trained Model

In [ ]:
# Load the trained model
model_path = MODEL_DIR / 'best_model.keras'
model = tf.keras.models.load_model(str(model_path))

print(f"Model loaded from: {model_path}")
print(f"\nModel Architecture:")
print(f"  Total Parameters: {model.count_params():,}")
trainable = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
print(f"  Trainable Parameters: {trainable:,}")

In [ ]:
# Model summary
model.summary()

## 5. Training History

In [ ]:
# Load training history
history_path = MODEL_DIR / 'training_history.json'
with open(history_path, 'r') as f:
    history = json.load(f)

epochs_trained = len(history['accuracy'])
best_val_acc = max(history['val_accuracy'])
best_epoch = history['val_accuracy'].index(best_val_acc) + 1

print(f"Training History Loaded")
print(f"  Epochs Trained: {epochs_trained}")
print(f"  Best Validation Accuracy: {best_val_acc*100:.2f}% (Epoch {best_epoch})")
print(f"  Final Training Accuracy: {history['accuracy'][-1]*100:.2f}%")
print(f"  Final Validation Accuracy: {history['val_accuracy'][-1]*100:.2f}%")

In [ ]:
# Plot training history
epochs_range = range(1, len(history['accuracy']) + 1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(epochs_range, history['accuracy'], 'b-o', label='Training', markersize=3, linewidth=1.5)
axes[0].plot(epochs_range, history['val_accuracy'], 'r-s', label='Validation', markersize=3, linewidth=1.5)
axes[0].scatter([best_epoch], [best_val_acc], color='green', s=200, zorder=5, marker='*',
                label=f'Best: {best_val_acc*100:.2f}%')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(epochs_range, history['loss'], 'b-o', label='Training', markersize=3, linewidth=1.5)
axes[1].plot(epochs_range, history['val_loss'], 'r-s', label='Validation', markersize=3, linewidth=1.5)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# Calculate gap
final_train_acc = history['accuracy'][-1]
final_val_acc = history['val_accuracy'][-1]
gap = abs(final_train_acc - final_val_acc)

plt.suptitle(f'Training History (Train-Val Gap: {gap*100:.2f}%)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(MODEL_DIR / 'training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nTraining Analysis:")
print(f"  Final Training Accuracy: {final_train_acc*100:.2f}%")
print(f"  Final Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"  Train-Val Gap: {gap*100:.2f}%")
if gap > 0.15:
    print("  Status: OVERFITTING detected!")
elif gap < 0.05:
    print("  Status: Good generalization")
else:
    print("  Status: Acceptable generalization")

## 6. Load Test Dataset

In [ ]:
# Create test dataset from TF structure
TF_DATA_DIR = MODEL_DIR / 'tf_data'

test_ds = tf.keras.utils.image_dataset_from_directory(
    str(TF_DATA_DIR / 'test'),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

detected_classes = test_ds.class_names
print(f"Test dataset loaded")
print(f"Detected classes: {detected_classes}")

In [ ]:
# Show sample test images
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
fig.suptitle('Sample Test Images', fontsize=14, fontweight='bold')

for images, labels in test_ds.take(1):
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            ax.imshow(images[i].numpy().astype('uint8'))
            cls_name = detected_classes[labels[i]]
            color = 'red' if 'mite' in cls_name else 'green'
            ax.set_title(cls_name, fontsize=10, color=color)
            ax.axis('off')

plt.tight_layout()
plt.show()

## 7. Model Evaluation on Test Set

In [ ]:
# Preprocessing for evaluation
normalization = tf.keras.layers.Rescaling(1./127.5, offset=-1)

def preprocess(image, label):
    image = normalization(image)
    return image, label

test_ds_prep = test_ds.map(preprocess).prefetch(tf.data.AUTOTUNE)

# Get predictions
print("Running predictions on test set...")
y_true = []
y_pred_probs = []

for images, labels in test_ds_prep:
    preds = model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred_probs.extend(preds.flatten())

y_true = np.array(y_true)
y_pred_probs = np.array(y_pred_probs)

print(f"Test samples: {len(y_true)}")
print(f"Class distribution: {dict(zip(*np.unique(y_true, return_counts=True)))}")

In [ ]:
# Load model info for optimal threshold
info_path = MODEL_DIR / 'model_info.json'
with open(info_path, 'r') as f:
    model_info = json.load(f)

OPTIMAL_THRESHOLD = model_info.get('threshold', 0.45)
print(f"Using optimal threshold: {OPTIMAL_THRESHOLD}")

# Apply threshold
y_pred = (y_pred_probs > OPTIMAL_THRESHOLD).astype(int)

In [ ]:
# Classification Report
print("="*60)
print("CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_true, y_pred, target_names=detected_classes, digits=4))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=detected_classes, yticklabels=detected_classes, annot_kws={'size': 20})
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Normalized
cm_norm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + 1e-10)
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues', ax=axes[1],
            xticklabels=detected_classes, yticklabels=detected_classes, annot_kws={'size': 16})
axes[1].set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.savefig(MODEL_DIR / 'confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate metrics
from sklearn.metrics import precision_recall_fscore_support

accuracy = np.mean(y_true == y_pred)
p, r, f1, support = precision_recall_fscore_support(y_true, y_pred, average=None)
macro_f1 = np.mean(f1)

# Performance visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Overall metrics
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
metrics_values = [accuracy, np.mean(p), np.mean(r), macro_f1]
colors_metrics = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']

bars = axes[0].bar(metrics_names, metrics_values, color=colors_metrics, edgecolor='black')
axes[0].set_ylim([0, 1.1])
axes[0].set_title('Overall Model Performance', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Score')
for bar, val in zip(bars, metrics_values):
    axes[0].text(bar.get_x() + bar.get_width()/2, val + 0.02, f'{val:.2%}', ha='center', fontweight='bold')

# Per-class metrics
x = np.arange(len(detected_classes))
width = 0.25
axes[1].bar(x - width, p, width, label='Precision', color='#3498db', edgecolor='black')
axes[1].bar(x, r, width, label='Recall', color='#2ecc71', edgecolor='black')
axes[1].bar(x + width, f1, width, label='F1-Score', color='#e74c3c', edgecolor='black')
axes[1].set_ylim([0, 1.1])
axes[1].set_title('Per-Class Performance', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(detected_classes)
axes[1].legend()
axes[1].set_ylabel('Score')

plt.tight_layout()
plt.savefig(MODEL_DIR / 'performance_metrics.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Madam's Requirements Validation

In [ ]:
print("="*60)
print("MADAM'S REQUIREMENTS VALIDATION")
print("="*60)

all_pass = True
TOLERANCE = 0.10  # 10% tolerance

# Requirement 1: P/R/F1 balanced per class
print("\n[1] P/R/F1 should be close for EACH class (gap < 10%)")
print("-"*60)
for i, cls in enumerate(detected_classes):
    pr_gap = abs(p[i] - r[i])
    status = "PASS" if pr_gap < TOLERANCE else "FAIL"
    if pr_gap >= TOLERANCE:
        all_pass = False
    print(f"  {cls}:")
    print(f"    Precision: {p[i]*100:.2f}%")
    print(f"    Recall:    {r[i]*100:.2f}%")
    print(f"    F1-Score:  {f1[i]*100:.2f}%")
    print(f"    P-R Gap:   {pr_gap*100:.2f}% [{status}]")

# Requirement 2: F1 similar across classes
print("\n[2] F1-Scores should be similar across classes (diff < 10%)")
print("-"*60)
f1_diff = abs(f1[0] - f1[1]) if len(f1) > 1 else 0
status = "PASS" if f1_diff < TOLERANCE else "FAIL"
if f1_diff >= TOLERANCE:
    all_pass = False
for i, cls in enumerate(detected_classes):
    print(f"  {cls} F1: {f1[i]*100:.2f}%")
print(f"  F1 Difference: {f1_diff*100:.2f}% [{status}]")

# Requirement 3: Accuracy close to F1
print("\n[3] Accuracy should be close to F1-Score (diff < 10%)")
print("-"*60)
acc_f1_diff = abs(accuracy - macro_f1)
status = "PASS" if acc_f1_diff < TOLERANCE else "FAIL"
if acc_f1_diff >= TOLERANCE:
    all_pass = False
print(f"  Accuracy:  {accuracy*100:.2f}%")
print(f"  Macro F1:  {macro_f1*100:.2f}%")
print(f"  Difference: {acc_f1_diff*100:.2f}% [{status}]")

# Requirement 4: No overfitting
print("\n[4] Train-Val gap should be small (no overfitting, gap < 15%)")
print("-"*60)
train_val_gap = abs(final_train_acc - final_val_acc)
status = "PASS" if train_val_gap < 0.15 else "FAIL"
if train_val_gap >= 0.15:
    all_pass = False
print(f"  Training Accuracy:   {final_train_acc*100:.2f}%")
print(f"  Validation Accuracy: {final_val_acc*100:.2f}%")
print(f"  Gap: {train_val_gap*100:.2f}% [{status}]")

print("\n" + "="*60)
if all_pass:
    print("ALL REQUIREMENTS PASSED!")
else:
    print("SOME REQUIREMENTS NEED ATTENTION")
print("="*60)

## 9. Final Summary

In [ ]:
print("="*60)
print("TRAINING SUMMARY")
print("="*60)

print(f"\n--- Model ---")
print(f"  Architecture: MobileNetV2 (Transfer Learning)")
print(f"  Input Size: {IMG_SIZE[0]}x{IMG_SIZE[1]}x3")
print(f"  Threshold: {OPTIMAL_THRESHOLD}")

print(f"\n--- Dataset (NO DATA LEAKAGE) ---")
print(f"  Training: {sum(counts['train'].values())} images (with augmentations)")
print(f"  Validation: {sum(counts['validation'].values())} images (originals only)")
print(f"  Test: {sum(counts['test'].values())} images (originals only)")

print(f"\n--- Training ---")
print(f"  Epochs: {epochs_trained}")
print(f"  Best Epoch: {best_epoch}")
print(f"  Best Val Accuracy: {best_val_acc*100:.2f}%")
print(f"  Train-Val Gap: {train_val_gap*100:.2f}%")

print(f"\n--- Test Results ---")
print(f"  Test Accuracy: {accuracy*100:.2f}%")
print(f"  Macro F1-Score: {macro_f1*100:.2f}%")

print(f"\n--- Per-Class Results ---")
for i, cls in enumerate(detected_classes):
    print(f"  {cls}:")
    print(f"    Precision: {p[i]*100:.2f}%")
    print(f"    Recall: {r[i]*100:.2f}%")
    print(f"    F1-Score: {f1[i]*100:.2f}%")

print(f"\n--- Files ---")
print(f"  Model: {MODEL_DIR / 'best_model.keras'}")
print(f"  Info: {MODEL_DIR / 'model_info.json'}")

print("\n" + "="*60)
if all_pass:
    print("ALL MADAM'S REQUIREMENTS: PASSED")
else:
    print("SOME REQUIREMENTS NEED ATTENTION")
print("="*60)